In [118]:
import openai
import streamlit as st
import time
from modules import llm,cache_processing
from modules_ import db
openai.api_key = st.secrets["api_secret"]
# openai.api_base = "http://xx.xx.xx.xx:1234/v1"
# openai.api_key = ""
def chat_with_openai(prompt, st, model="gpt-3.5-turbo-1106", max_tokens=500, temperature=0.0, delay_time=0.01, num_retries=2):

    start = time.time()
    answer = ""
    full_response = ""
    retry_count = 0
    system_message = "A Data Engineer. You follow an approved plan. Generate the initial SQL based on the requirements provided. Send it to the Sr Data Analyst to be executed."
    DEFAULT_SYSTEM_MESSAGE = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply "TERMINATE" in the end when everything is done.
    """
    while retry_count <= num_retries:
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {
                        "content": {system_message}, 
                        "role": "system",
                        
                        "role": "user",
                        "content": f'{prompt}',
                    }
                ],
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
                max_tokens=max_tokens,
                temperature=temperature,
                stream=True,
                request_timeout=3,
            )
            
            placeholder = st.empty()
            for chunk in response:
                # print(answer, end="", flush=True)
                placeholder.markdown(full_response)
                full_response += answer
                event_time = time.time() - start
                event_text = chunk.choices[0].delta
                if "content" in event_text:
                    answer = event_text["content"]
            
            return full_response
        
        except Exception as e:
            retry_count += 1
            print(f"Retry attempt {retry_count} after API error: {e}")
            time.sleep(delay_time)
    
    return "Failed to get response after multiple retries"


In [119]:
START_SQL_DETERMINE = "--- SQL Query ---"
END_SQL_DETERMINE = "--- End SQL Query ---"

In [120]:

cache = cache_processing.CacheHandle()
cache.connect_db()

raw_cache = cache.get_raw_cache()
print(raw_cache[:5])

Connect db success!
False
No new cache in raw cache
[(1, 'Hiện tại đang có bao nhiêu cơ hội?', 'SELECT count(id) FROM public.crm_lead', 1, '2023-10-20 10:45:50.680492'), (3, 'Cơ hội nào có xác suất thành công cao nhất?', 'SELECT name, probability FROM public.crm_lead where probability = (select max(probability) from public.crm_lead)', 1, '2023-10-20 10:47:33.471420'), (4, 'Cơ hội nào có xác suất thành công thấp nhất?', 'SELECT name, probability FROM public.crm_lead where probability = (select min(probability) from public.crm_lead)', 1, '2023-10-20 10:47:33.474721'), (5, 'Những cơ hội dự kiến kết thúc trong tuần này?', 'SELECT name, date_deadline FROM public.crm_lead where EXTRACT(YEAR FROM date_deadline) = EXTRACT(YEAR FROM NOW()) AND EXTRACT(WEEK FROM date_deadline) = EXTRACT(WEEK FROM NOW())', 1, '2023-10-20 10:47:33.478085'), (6, 'Những cơ hội nào đã thất bại?', 'SELECT name FROM public.crm_lead where lost_reason_id is not NULL', 1, '2023-10-20 10:47:33.481668')]


In [121]:
#convert raw_cache to dict type Q: A:
raw_cache_dict = []
for item in raw_cache:
    raw_cache_dict.append({"Question":item[1],"Answer":item[2]})
print(raw_cache_dict)


[{'Question': 'Hiện tại đang có bao nhiêu cơ hội?', 'Answer': 'SELECT count(id) FROM public.crm_lead'}, {'Question': 'Cơ hội nào có xác suất thành công cao nhất?', 'Answer': 'SELECT name, probability FROM public.crm_lead where probability = (select max(probability) from public.crm_lead)'}, {'Question': 'Cơ hội nào có xác suất thành công thấp nhất?', 'Answer': 'SELECT name, probability FROM public.crm_lead where probability = (select min(probability) from public.crm_lead)'}, {'Question': 'Những cơ hội dự kiến kết thúc trong tuần này?', 'Answer': 'SELECT name, date_deadline FROM public.crm_lead where EXTRACT(YEAR FROM date_deadline) = EXTRACT(YEAR FROM NOW()) AND EXTRACT(WEEK FROM date_deadline) = EXTRACT(WEEK FROM NOW())'}, {'Question': 'Những cơ hội nào đã thất bại?', 'Answer': 'SELECT name FROM public.crm_lead where lost_reason_id is not NULL'}, {'Question': "Lý do cơ hội 'AI Chatbot' thất bại là gì?", 'Answer': "SELECT cl.name, lr.name FROM public.crm_lead cl join public.crm_lost_rea

In [122]:
db = db.PostgresDB()
db.connect_with_url(st.secrets["DB_URL"])


In [123]:
table_defines = ['crm_lead','crm_lost_reason','crm_tag', 'crm_tag_rel', 'crm_stage','mail_activity','res_partner', 'res_users', 'res_company', 'utm_campaign', 'utm_medium', 'utm_source']

# print out the table definitions
table_definitions = db.get_multiple_table_definitions_for_prompt(table_defines)

local

In [124]:
# template_da = f"""you are a data analyst. you extract the main key word to retrieve the data from the database. you use the data to create a simple report that represent the main key in question. you send the report to the data engineer to be create and executed the query. So your task is really important. Answer report follow REPORT FORMAT.

# --------REPORT FORMAT--------
# 1. Question (User question): {{}}
# 2. Keywords
# 3. Short explain way to create the query.
# 4. Tips: for create efficient and using a wildcard or a pattern matching operator in SQL queries. allows for a more flexible search by matching any sequence of characters. It's very handy for searching when the exact syntax or spelling might not be known but a general pattern is. But still make sure that the query is not too broad and focus on the main key. For instance, using ILIKE '%key%' in a query would match any records where the specified field contains the substring 'key', regardless of what comes before or after it. This flexibility makes the query more tolerant to variations or potential mistakes in syntax or spelling.
# -----------------------------
# """

In [125]:
template_da = f"""you are a data analyst. you extract the main key word to retrieve the data from the database. you use the data to create a simple report that represent the main key in question. you send the report to the data engineer to be create and executed the query. So your task is really important. Answer report follow REPORT FORMAT.

--------DATA ANALYST REPORT--------
1. Question (User question): {{}}
2. Keywords 
3. Explain user question to Data Enginner more clearly understand
4. Tips:For instance, using ILIKE '%key%' in a query would match any records where the specified field contains the substring 'key', regardless of what comes before or after it. This flexibility makes the query more tolerant to variations or potential mistakes in syntax or spelling.
5. Pseudo SQL Query
-----------------------------
"""

In [126]:
# prompt_da = f"""you are a data analyst. you extract the main key word to retrieve the data from the database. you use the data to create a simple report that represent the main key in question. you send the report to the data engineer to be create and executed the query. So your task is really important. Answer report follow REPORT FORMAT.

# --------REPORT FORMAT--------
# 1. Question (User question): {{}}
# 2. Keywords
# 3. Explain raw text data for data engineer understand to create the query that match the question.
# 4. Tips: for create efficient and using a wildcard or a pattern matching operator in SQL queries. allows for a more flexible search by matching any sequence of characters. It's very handy for searching when the exact syntax or spelling might not be known but a general pattern is. But still make sure that the query is not too broad and focus on the main key. For instance, using ILIKE '%key%' in a query would match any records where the specified field contains the substring 'key', regardless of what comes before or after it. This flexibility makes the query more tolerant to variations or potential mistakes in syntax or spelling.
# -----------------------------

# Use these TABLE_DEFINITIONS to satisfy the report.

# TABLE_DEFINITIONS

# CREATE TABLE crm_lead (
#     lang_id integer,
#     campaign_id integer,
#     source_id integer,
#     medium_id integer,
#     message_main_attachment_id integer,
#     message_bounce integer,
#     user_id integer,
#     team_id integer,
#     company_id integer,
#     stage_id integer,
#     color integer,
#     recurring_plan integer,
#     partner_id integer,
#     title integer,
#     id integer NOT NULL,
#     state_id integer,
#     country_id integer,
#     lost_reason_id integer,
#     create_uid integer,
#     write_uid integer,
#     date_deadline date,
#     lead_properties jsonb,
#     expected_revenue numeric,
#     prorated_revenue numeric,
#     recurring_revenue numeric,
#     recurring_revenue_monthly numeric,
#     recurring_revenue_monthly_prorated numeric,
#     active boolean,
#     date_closed timestamp without time zone,
#     date_action_last timestamp without time zone,
#     date_open timestamp without time zone,
#     date_last_stage_update timestamp without time zone,
#     date_conversion timestamp without time zone,
#     create_date timestamp without time zone,
#     write_date timestamp without time zone,
#     day_open double precision,
#     day_close double precision,
#     probability double precision,
#     automated_probability double precision,
#     iap_enrich_done boolean,
#     lead_mining_request_id integer,
#     reveal_id character varying,
#     phone_sanitized character varying,
#     email_normalized character varying,
#     email_cc character varying,
#     name character varying NOT NULL,
#     referred character varying,
#     type character varying NOT NULL,
#     priority character varying,
#     contact_name character varying,
#     partner_name character varying,
#     function character varying,
#     email_from character varying,
#     phone character varying,
#     mobile character varying,
#     phone_state character varying,
#     email_state character varying,
#     website character varying,
#     street character varying,
#     street2 character varying,
#     zip character varying,
#     city character varying,
#     description text
# );

# -- Top 3 rows sample:
# lang_id  campaign_id  source_id  medium_id  message_main_attachment_id  message_bounce  user_id  team_id  company_id  stage_id  color  recurring_plan  partner_id  title  id  state_id  country_id  lost_reason_id  create_uid  write_uid  date_deadline  lead_properties         expected_revenue  prorated_revenue                               recurring_revenue  recurring_revenue_monthly  recurring_revenue_monthly_prorated  active         date_closed                  date_action_last  date_open               date_last_stage_update  date_conversion  create_date  write_date  day_open                        day_close  probability  automated_probability  iap_enrich_done  lead_mining_request_id  reveal_id  phone_sanitized  email_normalized  email_cc   name  referred  type  priority  contact_name         partner_name  function                    email_from             
#      phone  mobile                      phone_state                 email_state  website  street  street2  zip   city  description
# 34       7            2          10         None                        0               2        1        1           1         0      None            22          None   86  None      None        3               2           2          063275674      dgaygjjgd@gmail.com     None              Dự án Tối ưu hóa Chuỗi Cung ứng                Olivia Clark       opportunity                1                                   Stella Wright  Công ty thời trang thủ công  Nhân Viên         dgaygjjgd@gmail.com     07362746823             063275674        None         correct     https://thoitranthcong.com.vn/  None       None         None                   None             2023-10-18              None       None             0.0               0.00       None  0.00      0.00  False     2023-10-20 07:27:22  None          2023-10-20 07:23:21.627775  2023-10-20 07:23:21.627775  None   2023-10-20 07:23:21.627775  2023-10-20 07:29:31.691793  0.0          0.0      0.0     70.83    None  None  None 

# 35       8            4          7          None                        0               2        1        1           1         0      None            22          None   86  None      5           None            2           2          0267576813     dgaygjjgd@gmail.com     None              Dự án Phát triển Trang web Thương mại điện tử  Quinn Turner       opportunity                2                                   Stella Wright  My Company                   Nhân Viên         dgaygjjgd@gmail.com     07362746823             0267576813       None         correct     https://www.odooai.cn           PVC        None         None                   hcm              2023-09-26              None       None             2686611.0         334483.07  None  0.00      0.00  True      None                 None          2023-10-20 07:29:31.691793  2023-10-20 07:29:31.691793  None   2023-10-20 07:29:31.691793  2023-11-06 03:02:25.064644  0.0          0.0      12.45   70.83    None  True  None 

# 20       None         None       None       None                        0               2        1        1           2         0      None            3           None   1   None      None        1               2           2          +10886617378   tanaha200002@gmail.com  None              AI Chatbot                                     None               opportunity                0                                   Administrator  None                         None              tanaha200002@gmail.com  +10886617378            None             None         correct     None                            None       None         None                   None             None                    None       None             0.0               0.00       None  0.00      0.00  False     2023-10-06 03:27:21  None          2023-10-06 03:20:03.731900  2023-10-06 03:20:03.731900  None   2023-10-06 03:20:03.731900  2023-10-06 03:27:21.007546  0.0          0.0      0.0     0.0      None  True  None 


# CREATE TABLE crm_lost_reason (
#     id integer NOT NULL,
#     create_uid integer,
#     write_uid integer,
#     name jsonb NOT NULL,
#     active boolean,
#     create_date timestamp without time zone,
#     write_date timestamp without time zone
# );

# -- Top 3 rows sample:
# id  create_uid  write_uid  name                                                                                     active  create_date                 write_date
# 3   1           1          {'en_US': 'Not enough stock', 'vi_VN': 'Không đủ tồn kho'}                               True    2023-10-05 07:27:54.802641  2023-10-05 07:27:54.802641
# 1   1           1          {'en_US': 'Too expensive', 'vi_VN': 'Quá đắt'}                                           True    2023-10-05 07:27:54.802641  2023-10-05 07:27:54.802641
# 2   1           1          {'en_US': "We don't have people/skills", 'vi_VN': 'Chúng ta không có nhân lực/kỹ năng'}  True    2023-10-05 07:27:54.802641  2023-10-05 07:27:54.802641

# CREATE TABLE crm_recurring_plan (
#     id integer NOT NULL,
#     number_of_months integer NOT NULL,
#     sequence integer,
#     create_uid integer,
#     write_uid integer,
#     name jsonb NOT NULL,
#     active boolean,
#     create_date timestamp without time zone,
#     write_date timestamp without time zone
# );

# -- Top 3 rows sample:
# id  number_of_months  sequence  create_uid  write_uid  name                                             active  create_date                 write_date
# 1   1                 10        1           1          {'en_US': 'Monthly', 'vi_VN': 'Hàng tháng'}      True    2023-10-05 07:27:54.802641  2023-10-05 07:27:54.802641
# 2   12                10        1           1          {'en_US': 'Yearly', 'vi_VN': 'Hàng năm'}         True    2023-10-05 07:27:54.802641  2023-10-05 07:27:54.802641
# 3   36                10        1           1          {'en_US': 'Over 3 years', 'vi_VN': 'Hơn 3 năm'}  True    2023-10-05 07:27:54.802641  2023-10-05 07:27:54.802641

# CREATE TABLE crm_stage (
#     write_date timestamp without time zone,
#     sequence integer,
#     team_id integer,
#     create_uid integer,
#     write_uid integer,
#     name jsonb NOT NULL,
#     id integer NOT NULL,
#     is_won boolean,
#     fold boolean,
#     create_date timestamp without time zone,
#     requirements text
# );

# -- Top 3 rows sample:
# write_date  sequence  team_id  create_uid  write_uid  name                                                       id    is_won  fold  create_date                 requirements
# 1           0         None     1           2          {'en_US': 'New', 'vi_VN': 'Mới'}                           None  None    None  2023-10-05 07:27:54.802641  2023-10-20 06:51:48.314712
# 5           1         None     2           2          {'en_US': 'Kiểm tra', 'vi_VN': 'Kiểm tra'}                 None  None    None  2023-10-20 06:51:48.213676  2023-10-20 06:51:51.670847
# 2           2         None     1           2          {'en_US': 'Qualified', 'vi_VN': 'Đã đánh giá chất lượng'}  None  None    None  2023-10-05 07:27:54.802641  2023-10-20 06:51:51.670847

# CREATE TABLE mail_activity (
#     calendar_event_id integer,
#     res_model_id integer NOT NULL,
#     res_id integer,
#     activity_type_id integer,
#     user_id integer NOT NULL,
#     request_partner_id integer,
#     recommended_activity_type_id integer,
#     previous_activity_type_id integer,
#     create_uid integer,
#     write_uid integer,
#     automated boolean,
#     create_date timestamp without time zone,
#     write_date timestamp without time zone,
#     id integer NOT NULL,
#     date_deadline date NOT NULL,
#     res_model character varying,
#     res_name character varying,
#     summary character varying,
#     note text
# );

# -- Top 3 rows sample:
# calendar_event_id  res_model_id  res_id  activity_type_id  user_id  request_partner_id  recommended_activity_type_id  previous_activity_type_id  create_uid  write_uid  automated  create_date         write_date                    id          date_deadline  res_model  res_name                    summary                     note
# 3                  402           29      4                 2        None                None                          None                       2           2          crm.lead   PHÒNG LAB ODOO      mmmmmm                        2023-10-23  None           None       2023-10-18 04:31:10.477366  2023-10-18 04:31:10.477366  None
# 4                  402           31      3                 2        None                None                          None                       2           2          crm.lead   cơ hội đổi mới 5.0  thảo luận các vấn đề đổi mới  2023-10-20  None           None       2023-10-20 08:06:19.779346  2023-10-20 08:06:19.779346  None
# 5                  402           31      1                 14       None                None                          None                       2           2          crm.lead   cơ hội đổi mới 5.0  thảo luận                     2023-10-23  None           None       2023-10-20 08:07:18.296660  2023-10-20 08:07:18.296660  None

# CREATE TABLE res_partner (
#     company_id integer,
#     title integer,
#     parent_id integer,
#     user_id integer,
#     state_id integer,
#     date date,
#     country_id integer,
#     partner_latitude numeric,
#     partner_longitude numeric,
#     active boolean,
#     employee boolean,
#     is_company boolean,
#     partner_share boolean,
#     write_date timestamp without time zone,
#     message_main_attachment_id integer,
#     message_bounce integer,
#     industry_id integer,
#     color integer,
#     commercial_partner_id integer,
#     signup_expiration timestamp without time zone,
#     calendar_last_notif_ack timestamp without time zone,
#     team_id integer,
#     partner_gid integer,
#     create_uid integer,
#     id integer NOT NULL,
#     write_uid integer,
#     create_date timestamp without time zone,
#     phone_sanitized character varying,
#     name character varying,
#     display_name character varying,
#     ref character varying,
#     lang character varying,
#     tz character varying,
#     vat character varying,
#     company_registry character varying,
#     website character varying,
#     function character varying,
#     type character varying,
#     street character varying,
#     street2 character varying,
#     zip character varying,
#     city character varying,
#     email character varying,
#     phone character varying,
#     mobile character varying,
#     commercial_company_name character varying,
#     company_name character varying,
#     comment text,
#     email_normalized character varying,
#     signup_token character varying,
#     signup_type character varying,
#     additional_info character varying
# );

# -- Top 3 rows sample:
# company_id  title  parent_id                   user_id                            state_id  date  country_id  partner_latitude  partner_longitude  active  employee  is_company  partner_share  write_date  message_main_attachment_id         message_bounce  industry_id  color         commercial_partner_id  signup_expiration  calendar_last_notif_ack  team_id  partner_gid  create_uid  id    write_uid  create_date  phone_sanitized      name          display_name  ref                               
#  lang  tz    vat   company_registry  website  function  type  street  street2  zip                         city  email  phone                mobile  commercial_company_name  company_name  comment              email_normalized  signup_token  signup_type  additional_info
# 22          None   2023-10-20 07:13:57.673089  Stella Wright                      None      None  2           None          
#     None               None    0         22          2              2           Stella Wright                      None            vi_VN        Asia/Bangkok  None                   None               None                     None     contact      None        None  None       None         dgaygjjgd@gmail.com  07362746823   None          None                              
#  None  None  None  None              None     True      None  False   True     2023-10-20 07:29:31.691793  None  0      dgaygjjgd@gmail.com  None    None                     None          2023-10-20 07:13:57  1                 None          None         07362746823
# 24          None   2023-10-20 07:17:23.041151  Công ty TNHH Xây Dựng và Thiết Kế  None      None  2           None          
#     None               None    0         24          2              2           Công ty TNHH Xây Dựng và Thiết Kế  None            vi_VN        Asia/Bangkok  None                   None               None                     None     contact      None        None  None       None         avasmith@gmail.com   +10886617378  None          Công ty TNHH Xây Dựng và Thiết Kế  None  None  None  None              None     True      None  True    True     2023-10-20 07:31:49.849015  None  0      avasmith@gmail.com   None    None                     None          2023-10-20 07:17:23  1                 None          None         +10886617378
# 10          None   2023-10-06 10:14:39.258083  Công ty cổ phần BETA MEDIA         None      None  None        None          
#     None               None    0         10          2              2           Công ty cổ phần BETA MEDIA         None            vi_VN        Asia/Bangkok  None                   None               None                     None     contact      None        None  None       None         betamedia@gmail.com  087468264     None          None                              
#  None  None  None  None              None     True      None  False   False    2023-10-20 07:42:11.189062  None  0      betamedia@gmail.com  None    None                     None          2023-10-06 10:14:39  None              None          None         087468264


# """


In [127]:
def da_report(query,prompt,st):
    final_prompt  = prompt.replace("{}",query)
    # print(f"------------------GET DA REPORT------------------")
    # print(f'{final_prompt}')
    # print(f"------------------END DA_REPORT------------------")
    da_report = chat_with_openai(final_prompt, st)
    return da_report



In [128]:
response_de_head = f""""
{{}}
\n
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
\n\nUse these TABLE_DEFINITIONS to satisfy the database SQL query as a PostgreSQL format.\n\n"""



response_de_tail = f"""\nRespond in this format RESPONSE_FORMAT. I need to be able to easily parse the sql query from your response. Also THINK as a user, if your are user, what default information you want to GET?   

RESPONSE_FORMAT


            --- SQL Query ---
            sql query exclusively as a raw text
            --- End SQL Query ---

"""

In [129]:
template_de = llm.add_cap_ref(
    response_de_head,
    table_definitions,
    response_de_tail,
    ""
)

In [130]:
print(template_de)

"
{}


When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.


Use these TABLE_DEFINITIONS to satisfy the database SQL query as a PostgreSQL format.

 Table crm_lead, colums = [*,
 campaign_id: integer -> column description: id from utm_campaign table,
 source_id: integer -> column description: id from utm_source table,
 medium_id: integer -> column description: id from utm_medium table,
 team_id: integer -> column description: id from crm_team table,
 company_id: integer -> column description: id from res_company table,
 stage_id: integer -> column description: id from crm_stage table,
 lost_reason_id: integer -> column description: id from crm_lost_reason table,
 date_deadline: date -> column description: Expected project end date of the opportunity,
 expected_revenue: numeric -> column description: Expected revenue of the opportunity,
 date_open: timestamp without time zone -> column description: Date when the opportunity was

In [131]:
def get_answer(da_report,template_de, st):
    query = f"\n\nFollow the DATA ANALYST REPORT of data analyst. You are a Data Engineer. You follow an approved plan. Generate the initial SQL be executed by the data analyst. Focus on Pseudo code SQL of REPORT and TABLE_DEFINITIONS below to correctly the SQL.\n\n{da_report}"
    # template_de = template_de.format(query)
    template_de = template_de.replace("{}",query)
    # print("Here is template_de: ",template_de)
    # print(f"\n------------------GET ANSWER------------------")
    # print(f'{template_de}')
    # print(f"------------------END GET ANSWER------------------")
    sql_ = ""
    try:
        full_repsonse = chat_with_openai(template_de, st)
        #split sql
        full_repsonse = full_repsonse.split(START_SQL_DETERMINE)[1].split(END_SQL_DETERMINE)[0]
        sql_ = full_repsonse
        # print("Here is full_repsonse: ",full_repsonse)
        from modules.db import PostgresDB
        db = PostgresDB()
        url = st.secrets["DB_URL"]
        db.connect_with_url(url)
        curr = db.conn.cursor()
        curr.execute(full_repsonse)
        result = curr.fetchall()
        st.write(result)

    except Exception as e:
        result = f"Failed to execute the query. Error: {e}"
        
    return full_repsonse,result


In [132]:
import random
def Groupchat(query,template_da,template_de,raw_cache_dict,st):
    #sample raw cache
    sample_cache = random.sample(raw_cache_dict, 50)
    sample_prompt = f"""\nHere is some sample you can learn to get the insight and context of our database structure.\n ---------EXAMPLE---------\n"""
    for item in sample_cache:
        sample_prompt += f"Q: {item['Question']}\nA: {item['Answer']}\n"
    sample_prompt += f"---------END EXAMPLE---------\n"
    query = f"{query}\n{sample_prompt}"
    report_da = da_report(query,template_da,st)
    sql,result = get_answer(report_da,template_de,st)
    max_try = 3
    while 'Failed' in result and max_try > 0:
        print(f"\nFailed {max_try} times\n")
        #give prompt to user to edit the query
        new_prompt = f"""\nYou generate the wrong syntax SQL query, please notice that. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try. Focus on YOUR PREVIOUS SQL WRONG to correct it.\n\n---------YOUR PREVIOUS SQL WRONG---------\n
        {sql}
        \n---------END PREVIOUS SQL WRONG---------\n
        \n---------YOUR ERROR---------\n
        {result}
        \n---------END ERROR---------\n
        \n---------USER QUESTION---------\n
        {query}
        \n---------END USER QUESTION---------\n
        \n\n
        """
        sql,result = get_answer(new_prompt,template_de,st)
        max_try -= 1
    return sql,result



query = "Hiện tại có bao nhiêu nhân viên"
sql, result = Groupchat(query,template_da,template_de,raw_cache_dict,st)

print(f"-------sQL--------")
print(sql)
print(f"-------result--------")
print(result)
    

-------sQL--------

SELECT COUNT(DISTINCT rp.name) AS employee_count  
FROM crm_lead cl  
JOIN res_users ru ON cl.user_id = ru.id  
JOIN res_partner rp ON ru.partner_id = rp.id  
WHERE rp.function ILIKE '%nhân viên%'

-------result--------
[(0,)]


In [133]:
print(sql)


SELECT COUNT(DISTINCT rp.name) AS employee_count  
FROM crm_lead cl  
JOIN res_users ru ON cl.user_id = ru.id  
JOIN res_partner rp ON ru.partner_id = rp.id  
WHERE rp.function ILIKE '%nhân viên%'



In [134]:
print(result)

[(0,)]


### EVALUATION

In [135]:
result_predict = []
for item in raw_cache_dict[:5]:
    query = item['Question']
    sql, result = Groupchat(query,template_da,template_de,raw_cache_dict,st)
    # print("-----------")
    result_predict.append({"Question":query,"Answer":result})


In [136]:
result_predict

[{'Question': 'Hiện tại đang có bao nhiêu cơ hội?', 'Answer': [(25,)]},
 {'Question': 'Cơ hội nào có xác suất thành công cao nhất?',
  'Answer': [('[DƯỢC HẢI ĐỨC] DỰ ÁN CRM', 100.0, Decimal('300000000.0'))]},
 {'Question': 'Cơ hội nào có xác suất thành công thấp nhất?',
  'Answer': [('Dự án Tối ưu hóa Chuỗi Cung ứng', 0.0)]},
 {'Question': 'Những cơ hội dự kiến kết thúc trong tuần này?', 'Answer': []},
 {'Question': 'Những cơ hội nào đã thất bại?',
  'Answer': [('Dự án Tối ưu hóa Chuỗi Cung ứng', 3),
   ('AI Chatbot', 1),
   ('Dự án Tối ưu hóa Chuỗi Cung ứng', 2),
   ('Phần mềm Quản lý Tài sản Doanh nghiệp', 3)]}]

In [110]:
import pandas as pd
gt_result = []
from modules.db import PostgresDB
db = PostgresDB()
url = st.secrets["DB_URL"]
db.connect_with_url(url)
curr = db.conn.cursor()


def format_answer(answer):
    df = pd.DataFrame(answer)
    df.columns = [desc[0] for desc in curr.description]
    return df


for idx,item in enumerate(result_predict):

    answer = curr.execute(raw_cache_dict[idx]['Answer'])
    gt = curr.fetchall()
    if len(gt) == 0:
        gt = 'No result'
    else:
        gt = format_answer(gt)

    gt_result.append({"Question":raw_cache_dict[idx]['Question'],"Answer":gt})



In [111]:
print(gt_result)

[{'Question': 'Hiện tại đang có bao nhiêu cơ hội?', 'Answer':    count
0     25}, {'Question': 'Cơ hội nào có xác suất thành công cao nhất?', 'Answer':                        name  probability
0  [DƯỢC HẢI ĐỨC] DỰ ÁN CRM        100.0
1            PHÒNG LAB ODOO        100.0
2    John Doe's opportunity        100.0}]


In [48]:
result_predict

[{'Question': 'Hiện tại đang có bao nhiêu cơ hội?', 'Answer': [(25,)]},
 {'Question': 'Cơ hội nào có xác suất thành công cao nhất?',
  'Answer': [('[DƯỢC HẢI ĐỨC] DỰ ÁN CRM', 100.0),
   ('PHÒNG LAB ODOO', 100.0),
   ("John Doe's opportunity", 100.0)]}]

[{'Question': 'Hiện tại đang có bao nhiêu cơ hội?',
  'Answer': 'SELECT count(id) FROM public.crm_lead'},
 {'Question': 'Cơ hội nào có xác suất thành công cao nhất?',
  'Answer': 'SELECT name, probability FROM public.crm_lead where probability = (select max(probability) from public.crm_lead)'},
 {'Question': 'Cơ hội nào có xác suất thành công thấp nhất?',
  'Answer': 'SELECT name, probability FROM public.crm_lead where probability = (select min(probability) from public.crm_lead)'},
 {'Question': 'Những cơ hội dự kiến kết thúc trong tuần này?',
  'Answer': 'SELECT name, date_deadline FROM public.crm_lead where EXTRACT(YEAR FROM date_deadline) = EXTRACT(YEAR FROM NOW()) AND EXTRACT(WEEK FROM date_deadline) = EXTRACT(WEEK FROM NOW())'},
 {'Question': 'Những cơ hội nào đã thất bại?',
  'Answer': 'SELECT name FROM public.crm_lead where lost_reason_id is not NULL'}]

In [114]:
raw_cache_dict[:5]

[{'Question': 'Hiện tại đang có bao nhiêu cơ hội?',
  'Answer': 'SELECT count(id) FROM public.crm_lead'},
 {'Question': 'Cơ hội nào có xác suất thành công cao nhất?',
  'Answer': 'SELECT name, probability FROM public.crm_lead where probability = (select max(probability) from public.crm_lead)'},
 {'Question': 'Cơ hội nào có xác suất thành công thấp nhất?',
  'Answer': 'SELECT name, probability FROM public.crm_lead where probability = (select min(probability) from public.crm_lead)'},
 {'Question': 'Những cơ hội dự kiến kết thúc trong tuần này?',
  'Answer': 'SELECT name, date_deadline FROM public.crm_lead where EXTRACT(YEAR FROM date_deadline) = EXTRACT(YEAR FROM NOW()) AND EXTRACT(WEEK FROM date_deadline) = EXTRACT(WEEK FROM NOW())'},
 {'Question': 'Những cơ hội nào đã thất bại?',
  'Answer': 'SELECT name FROM public.crm_lead where lost_reason_id is not NULL'}]

In [137]:
def evaluation(test_data):
    final_result = []
    for row in test_data:
        
        gt = curr.execute(row['Answer'])
        gt = curr.fetchall()
        if len(gt) == 0:
            pass
        sql,predict = Groupchat(row['Question'],template_da,template_de,raw_cache_dict,st)
        final_result.append({"Question":row['Question'],"Ground Truth":gt,"Predict":predict})
    return final_result

list_ = evaluation(raw_cache_dict[:5])

    

Retry attempt 1 after API error: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out.


In [138]:
list_

[{'Question': 'Hiện tại đang có bao nhiêu cơ hội?',
  'Ground Truth': [(25,)],
  'Predict': [(25,)]},
 {'Question': 'Cơ hội nào có xác suất thành công cao nhất?',
  'Ground Truth': [('[DƯỢC HẢI ĐỨC] DỰ ÁN CRM', 100.0),
   ('PHÒNG LAB ODOO', 100.0),
   ("John Doe's opportunity", 100.0)],
  'Predict': [('[DƯỢC HẢI ĐỨC] DỰ ÁN CRM', 100.0),
   ('PHÒNG LAB ODOO', 100.0),
   ("John Doe's opportunity", 100.0)]},
 {'Question': 'Cơ hội nào có xác suất thành công thấp nhất?',
  'Ground Truth': [('Dự án Tối ưu hóa Chuỗi Cung ứng', 0.0),
   ('AI Chatbot', 0.0),
   ('Dự án Tối ưu hóa Chuỗi Cung ứng', 0.0),
   ('Phần mềm Quản lý Tài sản Doanh nghiệp', 0.0)],
  'Predict': [('Dự án Tối ưu hóa Chuỗi Cung ứng', 0.0)]},
 {'Question': 'Những cơ hội dự kiến kết thúc trong tuần này?',
  'Ground Truth': [],
  'Predict': []},
 {'Question': 'Những cơ hội nào đã thất bại?',
  'Ground Truth': [('Dự án Tối ưu hóa Chuỗi Cung ứng',),
   ('AI Chatbot',),
   ('Dự án Tối ưu hóa Chuỗi Cung ứng',),
   ('Phần mềm Quản lý

In [140]:
acc = 0
list_wrong = []
for row in list_:
    if row['Ground Truth'] == row['Predict']:
        acc += 1
    else:
        list_wrong.append(row)

print(acc/len(list_))

0.6


In [141]:
print(list_wrong)

[{'Question': 'Cơ hội nào có xác suất thành công thấp nhất?', 'Ground Truth': [('Dự án Tối ưu hóa Chuỗi Cung ứng', 0.0), ('AI Chatbot', 0.0), ('Dự án Tối ưu hóa Chuỗi Cung ứng', 0.0), ('Phần mềm Quản lý Tài sản Doanh nghiệp', 0.0)], 'Predict': [('Dự án Tối ưu hóa Chuỗi Cung ứng', 0.0)]}, {'Question': 'Những cơ hội nào đã thất bại?', 'Ground Truth': [('Dự án Tối ưu hóa Chuỗi Cung ứng',), ('AI Chatbot',), ('Dự án Tối ưu hóa Chuỗi Cung ứng',), ('Phần mềm Quản lý Tài sản Doanh nghiệp',)], 'Predict': [(34, 7, 2, 10, None, 0, 2, 1, 1, 1, 0, None, 22, None, 86, None, None, 3, 2, 2, '063275674', 'dgaygjjgd@gmail.com', None, 'Dự án Tối ưu hóa Chuỗi Cung ứng', 'Olivia Clark', 'opportunity', '1', 'Stella Wright', 'Công ty thời trang thủ công', 'Nhân Viên', 'dgaygjjgd@gmail.com', '07362746823', '063275674', None, 'correct', 'https://thoitranthcong.com.vn/', None, None, None, None, datetime.date(2023, 10, 18), None, None, Decimal('0.0'), Decimal('0.00'), None, Decimal('0.00'), Decimal('0.00'), Fals

In [88]:
#format answer from [()] to each column in df


format_answer(result_predict[1]['Answer'])



,name,probability
0,[DƯỢC HẢI ĐỨC] DỰ ÁN CRM,100.0


In [99]:
question ='Cơ hội nào có xác suất thành công cao nhất?'

gt = curr.execute(raw_cache_dict[0]['Answer'])
gt = curr.fetchall()



[(25,)]

In [96]:
raw_cache_dict[3]['Answer']

'SELECT name, date_deadline FROM public.crm_lead where EXTRACT(YEAR FROM date_deadline) = EXTRACT(YEAR FROM NOW()) AND EXTRACT(WEEK FROM date_deadline) = EXTRACT(WEEK FROM NOW())'

In [67]:
gt_answers = gt_result[1]['Answer']
for element in result_predict[1]['Answer']:
    print(element)
    if element in gt_answers:
        print("correct")
    else:
        print("wrong")

('[DƯỢC HẢI ĐỨC] DỰ ÁN CRM', 100.0)
correct


correct
